In [2]:
import argparse
import pandas as pd
from src.Utils.Dictionaries import team_index_current
from src.Utils.tools import *
from colorama import Fore, Style, init, deinit
from datetime import datetime
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import urllib.request
import re



from pycaret.classification import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
todays_games_url = 'https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2021/scores/00_todays_scores.json'
data_url = 'https://stats.nba.com/stats/leaguedashteamstats?' \
           'Conference=&DateFrom=&DateTo=&Division=&GameScope=&' \
           'GameSegment=&LastNGames=0&LeagueID=00&Location=&' \
           'MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&' \
           'PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&' \
           'PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&' \
           'Season=2021-22&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&' \
           'StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision='

In [5]:
def compile_prediction_data(games, team_stats_df, ml_odds_list, ou_spread_list):
    
    team_stats_dicts = team_stats_df.to_dict(orient='records')

    todays_game_dicts=[]

    for game in games:
        home_team=game['home_team']
        away_team=game['away_team']
        date=game['date']

        # Filter for betting values
        ou_dict=[dic for dic in ou_spread_list if (dic['home'] in home_team or dic['away'] in away_team) and dic['date'] in date][0]
        ml_dict=[dic for dic in ml_odds_list if (dic['home'] in home_team or dic['away'] in away_team) and dic['date'] in date][0]

        # Specify or use default values
        def_ou = ou_dict['over_under']
        
        ou = input(f'[{home_team} vs. {away_team}] Default total score ({def_ou}). Specify score: ')
        
        if not ou:
        
            ou = def_ou
        
        def_spread = ou_dict['spread']
        
        spread = input(f'[{home_team} vs. {away_team}] Default spread ({def_spread}). Specify spread: ')
        
        if not spread:
        
            spread = def_spread
            
        
        def_ml_home = ml_dict['home_odds']
        
        ml_home = input(f'[{home_team} vs. {away_team}] Default {home_team} ML Odds ({def_ml_home}). Specify odds: ')
        
        if not ml_home:
        
            ml_home = def_ml_home
        
        def_ml_away = ml_dict['away_odds']
        
        ml_away = input(f'[{home_team} vs. {away_team}] Default {away_team} ML Odds ({def_ml_away}). Specify odds: ')
        
        if not ml_away:
        
            ml_away = def_ml_away
            
        #Team info dict
        team_info_dict={
            'date': game['date'],
            'home': home_team,
            'away': away_team,
            'ou': ou,
            'spread': spread,
            'ml_home': ml_home,
            'ml_away': ml_away            

        }

        #Get stats
        home_team_stats_list=[dic for dic in team_stats_dicts if dic['TEAM_NAME']==home_team]
        away_team_stats_list=[dic for dic in team_stats_dicts if dic['TEAM_NAME']==away_team]

        #Filter for stats
        home_team_stats_dict={k+'_HOME': v for k, v in home_team_stats_list[0].items() if k not in ['TEAM_ID', 'CFID', 'CFPARAMS']}
        away_team_stats_dict={k+'_AWAY': v for k, v in away_team_stats_list[0].items() if k not in ['TEAM_ID', 'CFID', 'CFPARAMS']}

        #Combine stats and odds
        team_result_dict = {**team_info_dict, **home_team_stats_dict, **away_team_stats_dict}

        todays_game_dicts.append(team_result_dict)

    return todays_game_dicts

In [6]:
def scrap_todays_ml_odds():
    
    attempt=0
    
    while True:
        
        try:

            url="https://www.vegasinsider.com/nba/odds/las-vegas/money/"

            page = urllib.request.urlopen(url)
            
            break

        except:

            attempt+=1

            print(f'Page request failed. Attempt {attempt}.')
            
            sleep(3)
            
            if attempt>3:
                
                break
            
            else:
                
                continue        

    soup = BeautifulSoup(page, 'html.parser')
    
    odds_table_container = soup.find_all("table", class_="frodds-data-tbl")
    game_regex = re.compile('viCellBg')

    game_odds_list=[]
    for game_container in odds_table_container[0].findAll("tr", class_=game_regex):

        game_dict={}

        dt_str=game_container.findAll("span", class_="cellTextHot")[0].text

        month = dt_str[:2]
        day = dt_str[3:5]
        year = datetime.today().year
        time_str = dt_str[-8:]

        dt_string = f'{year}-{month}-{day} {time_str}'
        timestamp = datetime.strptime(dt_string, "%Y-%m-%d %I:%M %p")

        game_dict['date'] = dt_string[:10]
        game_dict['timestamp'] = timestamp

        team_containers = game_container.findAll("a", class_="tabletext")
        odds_containers = game_container.findAll("a", class_="cellTextNorm")[0].findAll('br')

        home_away_str = ['away','home']
        home_away_odds_str = ['away_odds','home_odds']

        for team_container, odds_container, ha_str, hao_str in zip(team_containers, odds_containers, home_away_str, home_away_odds_str):

            game_dict[ha_str] = team_container.text
            game_dict[hao_str] = int(odds_container.nextSibling)

        game_odds_list.append(game_dict)
    
    return game_odds_list

In [7]:
def scrap_todays_ou_spread():
    
    attempt=0
    
    while True:
        
        try:

            url="https://www.vegasinsider.com/nba/odds/las-vegas/"

            page = urllib.request.urlopen(url)
            
            break

        except:

            attempt+=1

            print(f'Page request failed. Attempt {attempt}.')
            
            sleep(3)
            
            if attempt>3:
                
                break
            
            else:
                
                continue  

    page = urllib.request.urlopen(url)

    soup = BeautifulSoup(page, 'html.parser')

    game_containers = soup.find_all("table", class_="frodds-data-tbl")
    game_regex = re.compile('viCellBg')

    spread_ou_list=[]
    for game_container in game_containers[0].findAll("tr", class_=game_regex):

        game_dict={}

        dt_str=game_container.findAll("span", class_="cellTextHot")[0].text

        month = dt_str[:2]
        day = dt_str[3:5]
        year = datetime.today().year
        time_str = dt_str[-8:]

        dt_string = f'{year}-{month}-{day} {time_str}'
        timestamp = datetime.strptime(dt_string, "%Y-%m-%d %I:%M %p")

        game_dict['date'] = dt_string[:10]
        game_dict['timestamp'] = timestamp

        team_containers = game_container.findAll("a", class_="tabletext")
        value_containers = game_container.findAll("a", class_="cellTextNorm")[0].findAll('br')

        home_away_str = ['away','home']

        for team_container, ha_str in zip(team_containers, home_away_str):

            game_dict[ha_str] = team_container.text

        str_strip = re.compile(r'[^\d.-]+')

        for value in value_containers:

            bet_value = value.nextSibling

            if "½" in bet_value:
                bet_value=bet_value.replace("½", ".5")

            # Determin if value is OU value
            if "u" in bet_value:

                ou_value=bet_value[:-3].replace("u", "")
                ou_value=str_strip.sub('', ou_value).split('-')[0]

            else:

                # Separate value
                spread_value = str_strip.sub('', bet_value).split('-')[1]

        game_dict['over_under']=float(ou_value)
        game_dict['spread']=float(spread_value)

        spread_ou_list.append(game_dict)
        
    return spread_ou_list

In [8]:
def predict(todays_game_dicts):
    
    pred_df = pd.DataFrame(todays_game_dicts)
    pred_df.columns = pred_df.columns.str.lower()

    #load models
    win_loss_model = load_model('win_loss_acc_72')
    ou_model = load_model('ou_cover_acc_56')
    
    #make predictions
    win_loss_prediction_df = predict_model(win_loss_model, data = pred_df, raw_score = True)
    ou_prediction_df = predict_model(ou_model, data = pred_df, raw_score = True)
    
    #Get ML expect values
    win_loss_results = get_expected_values(win_loss_prediction_df)
    ou_results = ou_prediction_df.to_dict(orient='records')
    
    return win_loss_results, ou_results

In [9]:
def color_selection(home_value,away_value,method='greater_less'):
    if method=='greater_less':
        
        if home_value>away_value:
            home_color=Fore.GREEN
            away_color=Fore.RED
        else:
            home_color=Fore.RED
            away_color=Fore.GREEN
            
        return home_color,away_color
    
    elif method=='positive_negative':
        
        if home_value>0:
            home_color=Fore.GREEN
        else: 
            home_color=Fore.RED
        if away_value>0:
            away_color=Fore.GREEN
        else: 
            away_color=Fore.RED
        return home_color,away_color

In [10]:
def print_results(win_loss_results, ou_results):
    
    for game in win_loss_results:
        
        date = game['date']
        home_team = game['home']
        away_team = game['away']
        home_team_ml_odds = game['ml_home']
        away_team_ml_odds = game['ml_away']
        ou = game['ou']
        home_team_w_prob = round(game['Score_W']*100,4)
        away_team_w_prob = round(game['Score_L']*100,4)
        home_w_color,away_w_color=color_selection(home_team_w_prob,
                                                  away_team_w_prob, 
                                                  method='greater_less')
        
        home_team_ml_expected_value = game['home_team_ml_expected_value']
        away_team_ml_expected_value = game['away_team_ml_expected_value']
        home_ev_color,away_ev_color=color_selection(home_team_ml_expected_value,
                                                  away_team_ml_expected_value,
                                                  method='positive_negative')
        
        ou_dict = [dic for dic in ou_results if dic['home']==home_team and dic['away']==away_team][0]
        over = round(ou_dict['Score_Over']*100,4)
        under = round(ou_dict['Score_Under']*100,4)
        ou_color,under_color=color_selection(over,
                                              under,
                                              method='greater_less')

        print(f'------ {date} [HOME] {Fore.BLUE }{home_team}{Style.RESET_ALL} vs. [AWAY] {Fore.MAGENTA}{away_team}{Style.RESET_ALL} ----')            
        print(f'[ML ODDS]           {home_team} ( {home_team_ml_odds} ) | {away_team} ( {away_team_ml_odds} )')
        print(f'[ML WIN CONFIDENCE] {home_team} ( {home_w_color}{home_team_w_prob}{Style.RESET_ALL} % ) | {away_team} ( {away_w_color}{away_team_w_prob}{Style.RESET_ALL} % )')
        print(f'[ML EXPECTED VALUE] {home_team} ( {home_ev_color}${home_team_ml_expected_value}{Style.RESET_ALL}  ) | {away_team} ( {away_ev_color}${away_team_ml_expected_value}{Style.RESET_ALL}  )')
        print(f'[OVER & UNDER]      OU ( {ou} ) OVER ( {ou_color}{over}{Style.RESET_ALL} % ) | UNDER ( {under_color}{under}{Style.RESET_ALL} % ) ')
        print('')

In [11]:
def to_pg(win_loss_results, ou_results):
    
    host = 'localhost'
    user = 'postgres'
    passwd = 4889
    db = 'nba_betting'
    
    engine = create_engine('postgresql+psycopg2://{}:{}@{}/{}'.format(user, passwd, host, db), echo = False)
    pd.DataFrame(win_loss_results).to_sql('win_loss_predictions', con=engine, if_exists = 'append', index=False)
    pd.DataFrame(ou_results).to_sql('over_under_predictions', con=engine, if_exists = 'append', index=False)

In [12]:
data = get_todays_games_json(todays_games_url)

In [13]:
games = create_todays_games(data)

In [14]:
team_stats = get_json_data(data_url)

In [15]:
team_stats_df = to_data_frame(team_stats)

In [16]:
ml_odds_list=scrap_todays_ml_odds()

In [17]:
ou_spread_list=scrap_todays_ou_spread()       

In [18]:
todays_game_dicts = compile_prediction_data(games, team_stats_df, ml_odds_list, ou_spread_list)

[Philadelphia 76ers vs. Orlando Magic] Default total score (213.0). Specify score: 
[Philadelphia 76ers vs. Orlando Magic] Default spread (11.5). Specify spread: 
[Philadelphia 76ers vs. Orlando Magic] Default Philadelphia 76ers ML Odds (-1000). Specify odds: 
[Philadelphia 76ers vs. Orlando Magic] Default Orlando Magic ML Odds (650). Specify odds: 
[Washington Wizards vs. Brooklyn Nets] Default total score (229.5). Specify score: 
[Washington Wizards vs. Brooklyn Nets] Default spread (2.0). Specify spread: 
[Washington Wizards vs. Brooklyn Nets] Default Washington Wizards ML Odds (110). Specify odds: 
[Washington Wizards vs. Brooklyn Nets] Default Brooklyn Nets ML Odds (-130). Specify odds: 
[Atlanta Hawks vs. Minnesota Timberwolves] Default total score (233.0). Specify score: 
[Atlanta Hawks vs. Minnesota Timberwolves] Default spread (2.0). Specify spread: 
[Atlanta Hawks vs. Minnesota Timberwolves] Default Atlanta Hawks ML Odds (-130). Specify odds: 
[Atlanta Hawks vs. Minnesota Tim

In [19]:
win_loss_results, ou_results = predict(todays_game_dicts)

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [20]:
print_results(win_loss_results, ou_results)

------ 2022-01-19 [HOME] Philadelphia 76ers vs. [AWAY] Orlando Magic ----
[ML ODDS]           Philadelphia 76ers ( -1000 ) | Orlando Magic ( 650 )
[ML WIN CONFIDENCE] Philadelphia 76ers ( 94.38 % ) | Orlando Magic ( 5.62 % )
[ML EXPECTED VALUE] Philadelphia 76ers ( $3.82  ) | Orlando Magic ( $-57.85  )
[OVER & UNDER]      OU ( 213.0 ) OVER ( 38.37 % ) | UNDER ( 60.54 % ) 

------ 2022-01-19 [HOME] Washington Wizards vs. [AWAY] Brooklyn Nets ----
[ML ODDS]           Washington Wizards ( 110 ) | Brooklyn Nets ( -130 )
[ML WIN CONFIDENCE] Washington Wizards ( 48.95 % ) | Brooklyn Nets ( 51.05 % )
[ML EXPECTED VALUE] Washington Wizards ( $2.8  ) | Brooklyn Nets ( $-9.68  )
[OVER & UNDER]      OU ( 229.5 ) OVER ( 28.98 % ) | UNDER ( 69.37 % ) 

------ 2022-01-19 [HOME] Atlanta Hawks vs. [AWAY] Minnesota Timberwolves ----
[ML ODDS]           Atlanta Hawks ( -130 ) | Minnesota Timberwolves ( 110 )
[ML WIN CONFIDENCE] Atlanta Hawks ( 62.9 % ) | Minnesota Timberwolves ( 37.1 % )
[ML EXPECTED VA

In [21]:
to_pg(win_loss_results, ou_results)